<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/10_19_22_SBERT%2B_BigGraph_Embeddings_NN_gossipcop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
#%%capture
!pip install -U sentence-transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The notebook for Gossipcop. (Similar one present for Politifact is already saved in the Drive and on Github) 

In [42]:
from pathlib import Path
import numpy as np
import pandas as pd
import re
import json

from sentence_transformers import SentenceTransformer 

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import wandb
from wandb.keras import WandbCallback
from IPython.display import clear_output

In [43]:
metrics = [accuracy_score, f1_score, precision_score, recall_score]

def get_name(score_func):
    return 'eval/' + score_func.__name__.split("_")[0]

# Load data

In [44]:
from google.colab import drive
drive.mount('/content/drive')

base_dir = Path("/content/drive/MyDrive/ResearchFND")
assert base_dir.exists()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
dataset_id = 'gossipcop'

In [46]:
import ast

converters = {"retweets":ast.literal_eval, "tweets":ast.literal_eval}
df = pd.read_csv(base_dir/f"{dataset_id}_agg.csv", converters=converters)
df.head()

,title,text,tweets,retweets,label,url,num_retweets,log_num_retweets,num_tweets,log_num_tweets
0,Kendall Kylie Jenner Jenner NOT Upset Up...,NaN,[],"[995423424741888001, 995461685166202880, 99987...",fake,NaN,3,1.386294,0,0.000000
1,Kim Kardashian Dethroned Dethroned By Khlo...,NaN,[],"[848843565027516416, 849030801970868224, 84884...",fake,NaN,3,1.386294,0,0.000000
2,Kim Kardashian Did NOT Hot Staffer Hot Sta...,NaN,[],"[940685393112064001, 977921622672920576, 94031...",fake,NaN,8,2.197225,0,0.000000
3,The Voice The Voice Team NOT Surprised Sur...,NaN,[],[],fake,NaN,0,0.000000,0,0.000000
4,Drake NOT Angelina Jolie s Toy Boy Toy Boy ...,NaN,"[{'id': 948630026496323585, 'text': 'Drake NOT...","[948022124626808832, 948630026496323585, 94801...",fake,NaN,18,2.944439,7,2.079442


# Using the embeddings for title and text from previous experiments

In [47]:
title_embedding_file = base_dir/f"{dataset_id}_sbert_title_embeddings.npy"

if title_embedding_file.exists():
    title_embeddings = np.load(title_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    titles = df.title.tolist()
    title_embeddings = model.encode(titles, show_progress_bar=True)
    np.save(title_embedding_file, title_embeddings)

In [48]:
text_embedding_file = base_dir/f"{dataset_id}_sbert_fulltext_embeddings.npy"

if text_embedding_file.exists():
    text_embeddings = np.load(text_embedding_file)
else:
    model_id = "all-mpnet-base-v2"
    model = SentenceTransformer(model_id)
    texts = (df.title + "\n" + df.text).tolist()
    text_embeddings = model.encode(texts, show_progress_bar=True)
    np.save(text_embedding_file, text_embeddings)

# Training - Saving the machine learning model specifications

In [49]:
import tensorflow as tf

In [50]:
def get_model(d_h=64, p_in:float=0.2, p_out:float=0.2):
    model = tf.keras.Sequential([
        tf.keras.layers.Dropout(p_in), # input dropout
        tf.keras.layers.Dense(d_h, activation='relu'),
        tf.keras.layers.Dropout(p_out),  
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  optimizer='adam',metrics=['accuracy'])
    return model

In [51]:
def train(fold_id, train_idx, test_idx, params):

    # setup
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    model = get_model()
    # training
    ckpt_filepath = f'ckpt/fold_{fold_id}'
    save_model_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=ckpt_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test), 
        batch_size = params['bs'] , epochs=params['epochs'],
        callbacks=[WandbCallback(), save_model_cb]
    )
    #evaluation
    model.load_weights(ckpt_filepath)
    logits = model.predict(X_test).squeeze()
    y_pred = (logits > 0.).astype(int)
    eval_results = {get_name(f):f(y_pred=y_pred, y_true=y_test) for f in metrics}
    wandb.log(eval_results)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(probs=None,
                            y_true=y_test, preds=y_pred,
                            class_names=["Fake", "Real"])})
    return history

### PyTorch_BigGraph embeddings

In [52]:
edim = 128
bg_embeddings = np.load(base_dir/f'{dataset_id}_pt_biggraph_article_embeddings_{edim}.npy')
idx = np.load(base_dir/f"{dataset_id}_pt_biggraph_article_idx_{edim}.npy")

In [53]:
graph_embeddings = np.zeros((text_embeddings.shape[0], edim))
graph_embeddings[idx] = bg_embeddings

In [54]:
params = {
    "epochs":600,
    "bs":128,
    # 'lr':1e-3
}

In [55]:
X = np.concatenate([
    text_embeddings, 
    df.log_num_retweets.to_numpy()[..., None], 
    df.log_num_tweets.to_numpy()[..., None],
    graph_embeddings], axis=1)
y = (df.label=="real").to_numpy().astype(int)

X.shape, y.shape

((19968, 898), (19968,))

In [56]:
skf = StratifiedKFold(shuffle=True, random_state=124)

Creating a group and logging all train, test data in wandb for better tracking and visualisation.

In [57]:
WANDB_ENTITY = 'saloniteam'
WANDB_PROJECTS = 'nofolds'

GROUP = f"{dataset_id}-fulltext-sbert+tw+ptbg128-mlp-e600"

for fold_id, (train_idx, test_idx) in enumerate(skf.split(X, y)):
    clear_output()
    with wandb.init(entity=WANDB_ENTITY, project=WANDB_ENTITY, group=GROUP, name=f"{GROUP}-{fold_id}") as run:
        train(fold_id, train_idx, test_idx, params)
    break

Epoch 1/600
121/125 [============================>.] - ETA: 0s - loss: 0.4685 - accuracy: 0.7777

wandb: Adding directory to artifact (/content/wandb/run-20221019_110651-262n1vwp/files/model-best)... Done. 0.1s


125/125 [==============================] - 3s 18ms/step - loss: 0.4669 - accuracy: 0.7791 - val_loss: 0.4055 - val_accuracy: 0.8205
Epoch 2/600
125/125 [==============================] - ETA: 0s - loss: 0.3812 - accuracy: 0.8339

wandb: Adding directory to artifact (/content/wandb/run-20221019_110651-262n1vwp/files/model-best)... Done. 0.1s


125/125 [==============================] - 2s 19ms/step - loss: 0.3812 - accuracy: 0.8339 - val_loss: 0.3818 - val_accuracy: 0.8282
Epoch 3/600
116/125 [==========================>...] - ETA: 0s - loss: 0.3642 - accuracy: 0.8433

...

wandb: Adding directory to artifact (/content/wandb/run-20221019_110651-262n1vwp/files/model-best)... Done. 0.1s


125/125 [==============================] - 1s 11ms/step - loss: 0.2347 - accuracy: 0.9050 - val_loss: 0.2896 - val_accuracy: 0.8753
Epoch 42/600
125/125 [==============================] - 1s 5ms/step - loss: 0.2274 - accuracy: 0.9099 - val_loss: 0.2915 - val_accuracy: 0.8823
Epoch 43/600
125/125 [==============================] - 1s 4ms/step - loss: 0.2266 - accuracy: 0.9081 - val_loss: 0.2936 - val_accuracy: 0.8681
Epoch 44/600
125/125 [==============================] - 1s 4ms/step - loss: 0.2276 - accuracy: 0.9079 - val_loss: 0.2918 - val_accuracy: 0.8806
Epoch 45/600
124/125 [============================>.] - ETA: 0s - loss: 0.2294 - accuracy: 0.9064

wandb: Adding directory to artifact (/content/wandb/run-20221019_110651-262n1vwp/files/model-best)... Done. 0.1s


125/125 [==============================] - 1s 10ms/step - loss: 0.2296 - accuracy: 0.9063 - val_loss: 0.2866 - val_accuracy: 0.8798
Epoch 46/600
125/125 [==============================] - 1s 5ms/step - loss: 0.2244 - accuracy: 0.9095 - val_loss: 0.2967 - val_accuracy: 0.8655
Epoch 47/600
125/125 [==============================] - 1s 5ms/step - loss: 0.2257 - accuracy: 0.9074 - val_loss: 0.2923 - val_accuracy: 0.8778
Epoch 48/600
125/125 [==============================] - 0s 4ms/step - loss: 0.2217 - accuracy: 0.9107 - val_loss: 0.2934 - val_accuracy: 0.8771
Epoch 49/600
125/125 [==============================] - 1s 4ms/step - loss: 0.2254 - accuracy: 0.9081 - val_loss: 0.2887 - val_accuracy: 0.8783
Epoch 50/600
125/125 [==============================] - 1s 4ms/step - loss: 0.2167 - accuracy: 0.9125 - val_loss: 0.2918 - val_accuracy: 0.8751
Epoch 51/600
125/125 [==============================] - 1s 4ms/step - loss: 0.2208 - accuracy: 0.9086 - val_loss: 0.2942 - val_accuracy: 0.8713
Epoc

accuracy,▁▂▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇█▇████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eval/accuracy,▁
eval/f1,▁
eval/precision,▁
eval/recall,▁
loss,█▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▇▇▇█▅██▇█▆▄▅▄▆▅▄▄▄▃▃▅▆▅▄▄▄▅▄▄▅▄▅▄▄▅▆▅▄
val_loss,▃▂▁▁▁▁▁▂▂▂▂▃▃▃▄▄▅▄▅▅▅▅▆▅▅▆▆▆▇▇▇▇▇▇█▇████
accuracy,0.96607
best_epoch,44
